In [31]:
import pandas as pd
import re
from helper.onemap import OneMapQuery, convert_second_to_time_with_s
from tqdm import tqdm
import time
from datetime import datetime
from vroom import Vehicle, Job, Input, TimeWindow

In [32]:
def get_time_in_seconds(time_str):
    """Convert HH:MM format to seconds"""
    try:
        hours, minutes = map(int, time_str.split(':'))
        return hours * 3600 + minutes * 60
    except:
        raise ValueError("Time must be in HH:MM format")

def get_vehicle_time_window(vehicle_num=None):
    """Get start and end time for a vehicle"""
    while True:
        try:
            vehicle_prefix = f" for Vehicle {vehicle_num}" if vehicle_num else ""
            start_time = input(f"Enter start time (HH:MM){vehicle_prefix}: ")
            end_time = input(f"Enter end time (HH:MM){vehicle_prefix}: ")
            start_seconds = get_time_in_seconds(start_time)
            end_seconds = get_time_in_seconds(end_time)
            if end_seconds <= start_seconds:
                print("Error: End time must be after start time")
                continue
            return start_seconds, end_seconds
        except ValueError as e:
            print(f"Error: {e}")

In [33]:
# Get number of vehicles
num_vehicles = int(input("Enter number of vehicles: "))

# Ask if all vehicles have same time window
same_time_window = input("Do all vehicles have the same time window? (yes/no): ").lower() == 'yes'

# Get time windows for vehicles
vehicle_time_windows = []
if same_time_window:
    print("\nEntering time window for ALL vehicles:")
    start_seconds, end_seconds = get_vehicle_time_window()
    vehicle_time_windows = [(start_seconds, end_seconds)] * num_vehicles
else:
    print("\nEntering individual time windows:")
    for i in range(num_vehicles):
        start_seconds, end_seconds = get_vehicle_time_window(i+1)
        vehicle_time_windows.append((start_seconds, end_seconds))


Entering individual time windows:


In [34]:
# Read the Excel file
df = pd.read_excel('store/data/job_with_time.xlsx')

# Convert job time windows to seconds
def convert_time_window_to_seconds(time_window):
    """Convert time window list ['HH:MM:SS', 'HH:MM:SS'] to seconds"""
    if isinstance(time_window, list) and len(time_window) == 2:
        start_time = datetime.strptime(time_window[0], '%H:%M:%S')
        end_time = datetime.strptime(time_window[1], '%H:%M:%S')
        start_seconds = start_time.hour * 3600 + start_time.minute * 60 + start_time.second
        end_seconds = end_time.hour * 3600 + end_time.minute * 60 + end_time.second
        return start_seconds, end_seconds
    return None, None

# Add time window in seconds to dataframe
df['time_window_seconds'] = df['time_window'].apply(convert_time_window_to_seconds)
print("\nJob data loaded with time windows:")
display(df)


Job data loaded with time windows:


,job_id,address,time_window,time_window_seconds
0,1,87 FLORA DRIVE HEDGES PARK CONDOMINIUM SINGAPO...,"['12:48:00', '17:32:00']","(None, None)"
1,2,48 FABER WALK WATERFRONT @ FABER SINGAPORE 128993,"['09:39:00', '13:19:00']","(None, None)"
2,3,32 CARPENTER STREET SINGAPORE 059911,"['20:01:00', '20:44:00']","(None, None)"
3,4,355 TANGLIN ROAD GRACE ASSEMBLY OF GOD CHURCH ...,"['08:44:00', '10:18:00']","(None, None)"
4,5,48 SAINT THOMAS WALK ESPADA SINGAPORE 238126,"['13:29:00', '15:08:00']","(None, None)"
5,6,11 MOUNT SOPHIA SOPHIA HILLS SINGAPORE 228461,"['13:07:00', '18:59:00']","(None, None)"
6,7,356 ALEXANDRA ROAD ALEXIS SINGAPORE 159949,"['17:18:00', '19:33:00']","(None, None)"
7,8,81 WEST COAST DRIVE HUNDRED TREES SINGAPORE 12...,"['20:46:00', '20:51:00']","(None, None)"
8,9,22 JALAN TENGGIRI D' SAVILLE SINGAPORE 428268,"['19:27:00', '21:24:00']","(None, None)"
9,10,61 SCIENCE PARK ROAD THE GALEN SINGAPORE 117525,"['16:09:00', '17:12:00']","(None, None)"


In [35]:
om = OneMapQuery()
depot_latlong = (1.31158791334051, 103.863375124429)  # CT Hub 2
all_locations = [depot_latlong]  # Start with depot

print("Processing locations...")
location_index_map = {}  # Map postal code to location index
for idx, row in tqdm(df.iterrows(), total=len(df)):
    address = row['address']
    postal_match = re.search(r'SINGAPORE (\d{6})', address)
    if postal_match:
        postal = postal_match.group(1)
        latlong = om.get_postal_latlong(postal)
        if latlong:
            if postal not in location_index_map:
                all_locations.append(latlong)
                location_index_map[postal] = len(all_locations) - 1
            df.loc[idx, 'location_index'] = location_index_map[postal]
    time.sleep(0.5)

print(f"\nUnique locations processed: {len(all_locations)-1}")

Processing locations...


100%|██████████| 30/30 [00:36<00:00,  1.23s/it]


Unique locations processed: 30


In [36]:
print("\nCalculating route matrices...")
duration_matrix, distance_matrix = om.get_route_matrices(all_locations)


Calculating route matrices...


In [37]:
# Create VROOM input
vroom_input = Input()

# Add vehicles with time windows
for i in range(num_vehicles):
    vehicle = Vehicle(
        id=i+1,
        start=0,
        end=0,
        time_window=TimeWindow(
            start=vehicle_time_windows[i][0],
            end=vehicle_time_windows[i][1]
        )
    )
    vroom_input.add_vehicle(vehicle)

# Add jobs with time windows
for _, row in df.iterrows():
    start_seconds, end_seconds = row['time_window_seconds']
    job = Job(
        id=row['job_id'],
        location=int(row['location_index']),
        time_windows=[TimeWindow(start=start_seconds, end=end_seconds)]
    )
    vroom_input.add_job(job)

In [38]:
vroom_input.set_durations_matrix(
    profile="car",
    matrix_input=duration_matrix
)
vroom_input.set_distances_matrix(
    profile="car",
    matrix_input=distance_matrix
)

In [39]:
solution = vroom_input.solve(exploration_level=5, nb_threads=4)

# Convert solution routes to a dataframe for easier analysis
route_df = solution.routes.copy()

print("\nOptimization Results:")
print(f"Total duration: {convert_second_to_time_with_s(solution.summary.duration)}")
print(f"Total distance: {solution.summary.distance} meters")


Optimization Results:
Total duration: 02:50:47
Total distance: 88783 meters


In [40]:
import folium

# Create a map centered on Singapore
m = folium.Map(location=[1.352083, 103.819839], zoom_start=12, tiles="cartodbpositron")

# Add depot marker with special styling
folium.CircleMarker(
    location=depot_latlong,
    popup='Depot (CT Hub 2)',
    tooltip='Depot',
    color='red',
    fill=True,
    fillColor='red',
    radius=10
).add_to(m)

In [41]:
# Group routes by vehicle
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    route_sequence = []
    
    # Get sequence of locations for this vehicle
    for step in vehicle_route.itertuples():
        route_sequence.append(all_locations[step.location_index])
    
    # Plot routes between consecutive points
    for i in range(len(route_sequence)-1):
        start = route_sequence[i]
        end = route_sequence[i+1]
        # Plot route with sequence numbers, using vehicle_id as color index
        om.plot_routes(start, end, m, vehicle_id, (i, i+1))

In [42]:
m

In [43]:
m.save('job_with_time.html')

# Get all job IDs from the solution (excluding start/end rows)
completed_jobs = route_df[route_df['type'] == 'job']['id'].tolist()

# Get all job IDs from our original dataframe
all_jobs = df['job_id'].tolist()

# Check for missing jobs
missing_jobs = set(all_jobs) - set(completed_jobs)

if len(missing_jobs) == 0:
    print("All jobs were fulfilled!")
else:
    print("Warning: Some jobs were not fulfilled!")
    print("Missing jobs:", sorted(list(missing_jobs)))
    
print("\nMap has been saved as 'optimized_route_with_time.html'")
print(f"Total vehicles used: {len(route_df['vehicle_id'].unique())}")
print(f"Total stops: {len(completed_jobs)}")

Missing jobs: [2, 8, 10]

Map has been saved as 'optimized_route_with_time.html'
Total vehicles used: 3
Total stops: 27


In [44]:
print("\nRoute Details per Vehicle:")
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    if not vehicle_route.empty:
        print(f"\nVehicle {vehicle_id}:")
        print(f"Start time: {convert_second_to_time_with_s(vehicle_route.iloc[0]['arrival'])}")
        print(f"End time: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['arrival'])}")
        print(f"Total duration: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['duration'])}")
        print(f"Total distance: {vehicle_route.iloc[-1]['distance']} meters")
        print(f"Number of stops: {len(vehicle_route[vehicle_route['type'] == 'job'])}")


Route Details per Vehicle:

Vehicle 1:
Start time: 08:15:00
End time: 09:12:09
Total duration: 00:57:09
Total distance: 28250 meters
Number of stops: 11

Vehicle 2:
Start time: 16:15:00
End time: 17:10:54
Total duration: 00:55:54
Total distance: 26303 meters
Number of stops: 10

Vehicle 3:
Start time: 20:15:00
End time: 21:12:44
Total duration: 00:57:44
Total distance: 34230 meters
Number of stops: 6


In [45]:
solution.routes

,vehicle_id,type,arrival,duration,setup,service,waiting_time,location_index,id,description,distance
0,1,start,29700,0,0,0,0,0,<NA>,,0
1,1,job,30087,387,0,0,0,29,29,,3344
2,1,job,30174,474,0,0,0,3,3,,3946
3,1,job,30257,557,0,0,0,20,20,,4300
4,1,job,30519,819,0,0,0,13,13,,6411
5,1,job,30738,1038,0,0,0,14,14,,7844
6,1,job,31060,1360,0,0,0,28,28,,10587
7,1,job,31757,2057,0,0,0,18,18,,18144
8,1,job,31913,2213,0,0,0,9,9,,19116
9,1,job,32199,2499,0,0,0,11,11,,21029
